# Sprint1 機械学習フロー

In [1]:
# 今回使用するライブラリ
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt

In [2]:
# データ読み込み
app_train = pd.read_csv('C:/Users/miyas/kaggle/application_train.csv')
app_test = pd.read_csv('C:/Users/miyas/kaggle/application_test.csv')

In [3]:
# データを軽くする関数
def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

In [4]:
light_train, NAlist = reduce_mem_usage(app_train)
light_test, NAlist = reduce_mem_usage(app_test)

Memory usage of properties dataframe is : 286.22705078125  MB
******************************
Column:  SK_ID_CURR
dtype before:  int64
dtype after:  uint32
******************************
******************************
Column:  TARGET
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  CNT_CHILDREN
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  AMT_INCOME_TOTAL
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  AMT_CREDIT
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  AMT_ANNUITY
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  AMT_GOODS_PRICE
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  REGION_POP

dtype after:  float32
******************************
******************************
Column:  ELEVATORS_MEDI
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  ENTRANCES_MEDI
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  FLOORSMAX_MEDI
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  FLOORSMIN_MEDI
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  LANDAREA_MEDI
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  LIVINGAPARTMENTS_MEDI
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  LIVINGAREA_MEDI
dtype before:  float64
dtype after:  float32
******************************
***********************

dtype after:  uint8
******************************
******************************
Column:  REG_REGION_NOT_WORK_REGION
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  LIVE_REGION_NOT_WORK_REGION
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  REG_CITY_NOT_LIVE_CITY
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  REG_CITY_NOT_WORK_CITY
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  LIVE_CITY_NOT_WORK_CITY
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  EXT_SOURCE_1
dtype before:  float64
dtype after:  float32
******************************
******************************
Column:  EXT_SOURCE_2
dtype before:  float64
dtype after:  float32
******************************
******

dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_10
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_11
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_12
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_13
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_14
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_15
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_DOCUMENT_16
dtype before:  int64
dtype after:  uint8
******************************
******************************
Column:  FLAG_D

In [5]:
# shapeの確認
print('TRAINshape:',light_train.shape)
print('TESTshape:',light_test.shape)

TRAINshape: (307511, 122)
TESTshape: (48744, 121)


### 今回は欠損値のある特徴量は削除

In [6]:
# trainとtestを結合して，欠損値のある特徴量を削除
light_train['train_test_FLG'] = 0
light_test['train_test_FLG'] = 1
t_t = pd.concat([light_train,light_test],sort=False)
t_t_drop = t_t.dropna(how='any',axis=1)
t_t_drop.shape

(356255, 116)

## オブジェクト型のデータを処理
* ラベルの個数が２個以下はエンコーディング）
* それ以上はダミー変数化（数値の大きさに意味を持たせたくないから）ランダムフォレストの場合はあまり気にしなくてもいいのかも．
* ダミー変数の1行目は消す（多重共線性）

In [7]:
# object型の個数
object_count = t_t_drop.select_dtypes('object').apply(pd.Series.nunique, axis = 0)
object_count

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
dtype: int64

In [8]:
from sklearn.preprocessing import LabelEncoder
# ラベルエンコーダーをインスタンス化
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in t_t_drop:
    if t_t_drop[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(t_t_drop[col].unique())) <= 2:
            # Train on the training data
            le.fit(t_t_drop[col])
            # Transform both training and test_data2ing data
            t_t_drop[col] = le.transform(t_t_drop[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)
print('Features shape: ', t_t_drop.shape)

C:\Users\miyas\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


3 columns were label encoded.
Features shape:  (356255, 116)


In [9]:
# one-hotエンコーディング（１列目は削除）
t_t_dummy = pd.get_dummies(t_t_drop,drop_first=True)
print('Features shape: ', t_t_dummy.shape)

Features shape:  (356255, 195)


In [10]:
# trainとtestに戻す
train_data = t_t_dummy[t_t_dummy['train_test_FLG'] == 0].drop(['train_test_FLG','SK_ID_CURR'],axis=1)
test_data = t_t_dummy[t_t_dummy['train_test_FLG'] == 1].drop(['train_test_FLG','SK_ID_CURR'],axis=1)

# 目的変数をもとに戻す
train_data['TARGET'] = app_train['TARGET']

# 特徴量と目的変数に分ける
X_train = train_data.drop('TARGET',axis=1).values
X_test = test_data.values
y_train = train_data['TARGET'].values

# データ数の確認ｔ
print('元データ：',app_train.shape)
print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_test:',X_test.shape)

元データ： (307511, 123)
X_train: (307511, 193)
y_train: (307511,)
X_test: (48744, 193)


# 【問題１】クロスバリデーション

In [11]:
# ランダムフォレストのインスタンス化
rf = RandomForestClassifier(criterion='gini',n_estimators=20,random_state=0,n_jobs=2)
# kfoldインスタンス化
kfold = StratifiedKFold(n_splits=10,random_state=0).split(X_train,y_train)

# AUC確認
scores = cross_val_score(estimator=rf,X=X_train,y=y_train,n_jobs=-1,scoring='roc_auc',cv=3)
scores

array([0.6595833 , 0.65200151, 0.65463885])

# 【問題２】グリッドサーチ
* n_estimators … 決定木の数
* 'n_jobs': [-1] … pcのコアを全部使う
* criterion … 不純度の算出方法

In [12]:
# パラメータ
params = {'n_estimators': [10,20], 'n_jobs': [-1],'criterion':['gini','entropy']}

# インスタンス化
rf = RandomForestClassifier(random_state=0)
gs = GridSearchCV(estimator=rf,
                  param_grid=params,
                  scoring='roc_auc',
                  cv=3)

In [13]:
# 学習
gs.fit(X_train,y_train)

# 結果
print('ベストスコア：',gs.best_score_)
print('パラメータ：',gs.best_params_)

ベストスコア： 0.6606412110198797
パラメータ： {'criterion': 'entropy', 'n_estimators': 20, 'n_jobs': -1}


In [14]:
# グリッドサーチせずにやってみる

In [15]:
# kfoldインスタンス化
kfold = StratifiedKFold(n_splits=10,random_state=0).split(X_train,y_train)
rf2 = RandomForestClassifier(random_state=0,n_estimators=500,criterion='entropy')
rf2.fit(X_train,y_train)

# AUC確認,n_estimators=500
scores = cross_val_score(estimator=rf2,X=X_train,y=y_train,n_jobs=-1,scoring='roc_auc',cv=3)
scores

array([0.73401115, 0.72888191, 0.73189297])

# 【問題3】Kernelからの調査

### １．ランダムフォレストの特徴量の重要度で特徴量を削る

In [16]:
# 特徴量の重要度
rf.fit(X_train,y_train)
s = pd.Series(rf.feature_importances_,
             index=train_data.drop('TARGET',axis=1).columns)
X_importance = s.sort_values(ascending=False)
X_importance

C:\Users\miyas\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


EXT_SOURCE_2                           0.054870
EXT_SOURCE_3                           0.041712
DAYS_BIRTH                             0.036952
DAYS_REGISTRATION                      0.036237
DAYS_ID_PUBLISH                        0.035870
AMT_ANNUITY                            0.033334
DAYS_LAST_PHONE_CHANGE                 0.032963
DAYS_EMPLOYED                          0.031851
AMT_CREDIT                             0.031820
AMT_INCOME_TOTAL                       0.028283
REGION_POPULATION_RELATIVE             0.027895
AMT_GOODS_PRICE                        0.027593
HOUR_APPR_PROCESS_START                0.025359
EXT_SOURCE_1                           0.021514
AMT_REQ_CREDIT_BUREAU_YEAR             0.016426
OBS_30_CNT_SOCIAL_CIRCLE               0.014608
OBS_60_CNT_SOCIAL_CIRCLE               0.013737
OWN_CAR_AGE                            0.013160
CNT_FAM_MEMBERS                        0.010948
TOTALAREA_MODE                         0.009666
LIVINGAREA_MEDI                        0

In [17]:
# パラメータ
params2 = {'n_estimators': [300,400], 'n_jobs': [-1],'criterion':['gini','entropy']}

# 重要度が上から20番目までのデータで学習
rf_X_train = train_data[X_importance[:20,].index].values

# インスタンス化
gs2 = GridSearchCV(estimator=rf,
                  param_grid=params2,
                  scoring='roc_auc',
                  cv=3)

In [18]:
# 学習
gs2.fit(rf_X_train,y_train)

# 結果
print('ベストスコア：',gs2.best_score_)
print('パラメータ：',gs2.best_params_)

ベストスコア： 0.7325170249646893
パラメータ： {'criterion': 'entropy', 'n_estimators': 400, 'n_jobs': -1}


### 分かったこと
* n_estimatorsは400でも500でもあまり変わらない
* ランダムフォレストの特徴量の重要度の高さで絞っても，あまり精度上がらなかった．

### このモデルをカーネルにあげてみる

In [19]:
# testデータの重要度が上から20番目までを削る
rf_X_test = test_data[X_importance[:20,].index].values

In [20]:
# テストデータで推測
y_pred = gs2.predict(rf_X_test)
# 正解率を格納
y_proba = gs2.predict_proba(rf_X_test)
y_proba

array([[0.9475, 0.0525],
       [0.9275, 0.0725],
       [0.96  , 0.04  ],
       ...,
       [0.98  , 0.02  ],
       [0.945 , 0.055 ],
       [0.7575, 0.2425]])

２列目のTARGET1の正解率を提出する

In [21]:
# 新しいデータフレームを作って，提出する列のみ残して，csvに吐き出す
df_out = pd.read_csv('C:/Users/miyas/kaggle/application_test.csv')
df_out['TARGET'] = y_proba[:,1]
df_out[['SK_ID_CURR','TARGET']].to_csv('submission_3.csv',index=False)

### ２．PCAで特徴量を削ってみる

In [22]:
# 標準化
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [23]:
# パイプラインStandardScaler
pipe_rf = Pipeline([['pca',PCA(random_state=0)],
                  ['rf',RandomForestClassifier(n_estimators=400,
                                               random_state=0,
                                               n_jobs=-1,
                                               criterion='entropy')]])

# パラメータ
parms_pipe = {'pca__n_components':[5,10]}

# グリッドサーチ
gs_pca = GridSearchCV(estimator=pipe_rf,
                  param_grid=parms_pipe,
                  scoring='roc_auc',cv=3)

In [24]:
gs_pca.fit(X_train_std,y_train)
# 結果
print('ベストスコア：',gs_pca.best_score_)
print('パラメータ：',gs_pca.best_params_)

ベストスコア： 0.679972580069683
パラメータ： {'pca__n_components': 10}


### ３．やったことまとめ
#### ①カテゴリデータの処理方法
* 順序特徴量はマッピング処理する
* 順序に意味のない特徴量はsklearnのLabelEncoderを使う．
* しかしクラスラベルが多いもの（今回は３個以上）も，同じように処理してしまうと，数字の大きさに意味が出てきてしまうので，pandasのget_dummiesを使う．
* この時，多重共線性の問題が出てくるので，出来上がった特徴量の１列を削除した．

#### ②データ容量を小さくする
* dtypes変換することでデータが軽くなる，らしい．．．

#### ③特徴量の重要度
* ランダムフォレストで特徴量の重要度にアクセスできる

#### ④パイプライン
* sklearnのpipelineを使うと複数の関数をつなぐことができる

#### ⑤PCAで次元削減

#### ⑥勾配ブースティング
#### ⑦Earry Stopping¶
#### ⑧ランダムサーチ